# Generate personalized messages

## 1) Imports

In [1]:
import sys
import os
import pandas as pd

# Add the src folder to the path
sys.path.append(os.path.abspath("../src"))

from data_transformation import get_partition_limits, split_dataframe_by_quantiles, split_dataframe_by_values
from message import get_response


## 2) Read Dataset

In [ ]:
file_path = "your_file_path_here.csv"  # Replace with your actual file path
df = pd.read_csv(file_path)
df.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


## 3) Divide dataset into n costumer groups

In [3]:
# Divide by gender
dfs_gender = split_dataframe_by_values(df, 'Gender')
print("Dataframes: ")
print("df:", df.shape)
for i,each_df in enumerate(dfs_gender):
    print(f"df{i+1}:", each_df.shape)

Dataframes: 
df: (400, 5)
df1: (196, 5)
df2: (204, 5)


In [4]:
# Divide by Age
age_quantiles = get_partition_limits(df, 'Age', 3)
print("Age quantiles:", age_quantiles)

dfs_age = []
for each_df in dfs_gender:
    # Extend list (adds individual values to the list)
    dfs_age.extend(split_dataframe_by_quantiles(each_df, 'Age', age_quantiles)) 

print("Dataframes: ")
print("df:", df.shape)
for i,each_df in enumerate(dfs_age):
    print(f"df{i+1}:", each_df.shape)

Age quantiles: [33. 41.]
Dataframes: 
df: (400, 5)
df1: (69, 5)
df2: (71, 5)
df3: (56, 5)
df4: (71, 5)
df5: (58, 5)
df6: (75, 5)


In [5]:
# Divide by Salary
salary_quantiles = get_partition_limits(df, 'EstimatedSalary', 3)
print("Salary quantiles:", salary_quantiles)

dfs = []
for each_df in dfs_age:
    dfs.extend(split_dataframe_by_quantiles(each_df, 'EstimatedSalary', salary_quantiles)) 

print("Dataframes: ")
print("df:", df.shape)
for i,each_df in enumerate(dfs):
    print(f"df{i+1}:", each_df.shape)

Salary quantiles: [52000. 80000.]
Dataframes: 
df: (400, 5)
df1: (26, 5)
df2: (21, 5)
df3: (22, 5)
df4: (17, 5)
df5: (38, 5)
df6: (16, 5)
df7: (20, 5)
df8: (18, 5)
df9: (18, 5)
df10: (27, 5)
df11: (19, 5)
df12: (25, 5)
df13: (13, 5)
df14: (35, 5)
df15: (10, 5)
df16: (31, 5)
df17: (7, 5)
df18: (37, 5)


## 4) Generate Messages

In [ ]:
API_KEY = "YOUR_API_KEY"

In [10]:
system_prompt = "You are a helpful assistant."
user_prompt = "Hey, are you there?"
get_response(API_KEY, system_prompt, user_prompt)

"Yes, I'm here! How can I assist you today?"

In [14]:
descriptions = []
general_prompt = "I have a store and I am promoting a discount for a specific product. I want to generate a message personalized to each category of costumers. "
general_prompt_final = " (Answer only the message)"
for df_group in dfs:
    gender = df_group['Gender'].iloc[0] if not df_group.empty else 'Unknown'
    age_min, age_max = df_group['Age'].min(), df_group['Age'].max()
    salary_min, salary_max = df_group['EstimatedSalary'].min(), df_group['EstimatedSalary'].max()
    desc = f"My costumer has gender {gender} age between {age_min} and {age_max} and salary between {salary_min} and {salary_max}."
    descriptions.append(general_prompt + desc + general_prompt_final)
descriptions

['I have a store and I am promoting a discount for a specific product. I want to generate a message personalized to each category of costumers. My costumer has gender Male age between 18 and 33 and salary between 15000 and 52000. (Answer only the message)',
 'I have a store and I am promoting a discount for a specific product. I want to generate a message personalized to each category of costumers. My costumer has gender Male age between 19 and 31 and salary between 55000 and 80000. (Answer only the message)',
 'I have a store and I am promoting a discount for a specific product. I want to generate a message personalized to each category of costumers. My costumer has gender Male age between 18 and 32 and salary between 81000 and 148000. (Answer only the message)',
 'I have a store and I am promoting a discount for a specific product. I want to generate a message personalized to each category of costumers. My costumer has gender Male age between 34 and 41 and salary between 20000 and 52

In [15]:
desc = descriptions[0]
system_prompt = "You are a helpful assistant."
user_prompt = desc
response = get_response(API_KEY, system_prompt, user_prompt)
print("Response:", response)

Response: "Hey there! As a valued young man in our community, we’re excited to offer you an exclusive discount on [product name]. Elevate your experience with this fantastic deal, perfectly tailored for you. Don’t miss out!"


In [16]:
system_prompt = "You are a helpful assistant."
responses = []

for desc in descriptions:
    user_prompt = desc
    response = get_response(API_KEY, system_prompt, user_prompt)
    print("Response:", response)
    print("-" * 50)  # Separator for readability
    responses.append(response)

Response: Hey there! 🤙 We know you love staying on top of trends, so we're excited to offer you an exclusive discount on [Product]! Upgrade your collection now and enjoy a special deal just for you. Don't miss out – this is your chance to grab it at a great price! 🛒✨
--------------------------------------------------
Response: "Hey there! As a valued customer, we’re excited to offer you an exclusive discount on [Product Name]. Elevate your style and experience with this special offer just for you. Don’t miss out—grab yours today and enjoy the savings!"
--------------------------------------------------
Response: "Hey there! Get ready to level up your style with our exclusive discount on [Product]! This is the perfect opportunity to grab something that complements your unique taste while saving you money. Hurry, this offer is just for a select group of trendsetters like you. Don't miss out!"
--------------------------------------------------
Response: "Hello! As a valued customer, we’re

In [17]:
with open("personalized_messages.txt", "w") as f:
    for df_group, message in zip(dfs, responses):
        gender = df_group['Gender'].iloc[0] if not df_group.empty else 'Unknown'
        age_min = df_group['Age'].min() if not df_group.empty else 'N/A'
        age_max = df_group['Age'].max() if not df_group.empty else 'N/A'
        salary_min = df_group['EstimatedSalary'].min() if not df_group.empty else 'N/A'
        salary_max = df_group['EstimatedSalary'].max() if not df_group.empty else 'N/A'
        f.write(f"Gender: {gender}, Age: {age_min}-{age_max}, Salary: {salary_min}-{salary_max}\n")
        f.write(f"Message: {message}\n")
        f.write("-" * 50 + "\n")